In [1]:
import os

In [2]:
file_sparse = "data/lausanne_center/sparse"
file_images = "data/lausanne_center/images"
file_scene = "data/lausanne_center"

In [3]:
image_undistorter_command = f"colmap image_undistorter --image_path {file_images} --input_path {file_sparse} --output_path {file_scene}"

print(image_undistorter_command)

colmap image_undistorter --image_path data/lausanne_center/images --input_path data/lausanne_center/sparse --output_path data/lausanne_center


Estimate depth

In [4]:
ratio = 4
os.makedirs("../logs", exist_ok=True)
downsample_command = f"python utils/image_downsample.py {file_images} --factor {ratio}"
print(f"{downsample_command} |& tee logs/downsample.log")

python utils/image_downsample.py data/lausanne_center/images --factor 4 |& tee logs/downsample.log


In [5]:
estimate_command = f"python utils/estimate_dataset_depths.py {file_scene} -d {ratio}"
print(f"{estimate_command} |& tee estimate_depths.log")

python utils/estimate_dataset_depths.py data/lausanne_center -d 4 |& tee estimate_depths.log


A. Train coarse model

In [6]:
coarse_file = "configs/lausanne_9000_coarse.yaml"
name_coarse = "lausanne_coarse_9000"

coarse_command = f"python main.py fit --config {coarse_file} --n {name_coarse}"
log = "tee logs/lausanne_coarse.log"
print(f"{coarse_command} |& {log}")

python main.py fit --config configs/lausanne_9000_coarse.yaml --n lausanne_coarse_9000 |& tee logs/lausanne_coarse.log


B. Model Partition and Data Assignment

In [14]:
name_train = "lausanne_center"
train_yaml = f"configs/lausanne_train.yaml"
partition_command = f"nohup python utils/partition_citygs.py --config_path {train_yaml} --force --origin auto"
log = "tee logs/partition.log"
print(f"{partition_command} |& {log}")

nohup python utils/partition_citygs.py --config_path configs/lausanne_train.yaml --force --origin auto |& tee logs/partition.log


C. Finetune model parallelly and merge

In [15]:
fine_tune = f"python utils/train_citygs_partitions.py -n {name_train} -p {name_train}"
print(f"nohup {fine_tune} |& tee logs/train_citygs_partitions.log")



merge_command = f"python utils/merge_citygs_ckpts.py outputs/{name_train}"
log = "tee logs/merge_citygs.log"
print(merge_command + f" |& {log}")

nohup python utils/train_citygs_partitions.py -n lausanne_center -p lausanne_center |& tee logs/train_citygs_partitions.log
python utils/merge_citygs_ckpts.py outputs/lausanne_center |& tee logs/merge_citygs.log


D. Evaluate Rendering Performance

In [17]:
test_command = f"python main.py test --config outputs/{name_train}/config.yaml --save_val --test_speed"
print(f"nohup {test_command} |& tee logs/test_citygs.log")

nohup python main.py test --config outputs/lausanne_center/config.yaml --save_val --test_speed |& tee logs/test_citygs.log


E. Mesh extraction and evaluation

In [18]:
mesh_command = f"python utils/gs2d_mesh_extraction.py outputs/{name_train} --voxel_size 0.01 --sdf_trunc 0.04"
print(f"nohup {mesh_command} |& tee logs/mesh_extraction.log")

nohup python utils/gs2d_mesh_extraction.py outputs/lausanne_center --voxel_size 0.01 --sdf_trunc 0.04 |& tee logs/mesh_extraction.log


F. Compression

In [19]:
compression_command = f"python tools/vectree_lightning.py --model_path outputs/{name_train} --save_path outputs/{name_train}/vectree --sh_degree 3 --gs_dim 3"
print(compression_command + " |& tee logs/compression.log")

python tools/vectree_lightning.py --model_path outputs/lausanne_center --save_path outputs/lausanne_center/vectree --sh_degree 3 --gs_dim 3 |& tee logs/compression.log


In [20]:
sh_script = f"/mnt/Data/yanns/CityGaussian/scripts/{name_train}.sh"
commands = [downsample_command,
              estimate_command,
              coarse_command,
              partition_command,
              fine_tune,
              merge_command,
              test_command,
              mesh_command,
              compression_command]

if not os.path.exists(sh_script):
    with open(sh_script, "w") as f:
        f.write("#!/bin/bash\n")
        f.write("set -e\n")  # Exit on error
        f.write("set -x\n")  # Print commands before executing them

with open(sh_script, "w") as f:
    for command in commands:
        f.write(command + "\n")
        f.write("echo 'Done: " + command + "'\n")





